**___ Packing**

Quantum Computing is one of the most promising but clumsy technologies of today's world. Though critical applications are emerging in a wide variety of fields, the inefficiency of modern quantum processing units (QPUs) is holding them back from a more widespread utilization. One proposed solution for fighting this inefficiency is to run numerous instances of a provided quantum circuit at the same time by allocating space for multiple quantum circuits into a single QPU. For example, a circuit that requires 5 qubits running on a QPU with 12 available qubits leaves 7 unused, when instead, two of those circuits could run simultaneously and leave only 2 unused - not unlike parallelism in classical CPUs. 

Unfortunately, the solution is not as simple as fitting as many circuits onto a single QPU as possible. 

Due to error prone qubits with poor connections, it must be considered that some qubits are better suited for a task than other qubits. Additionally, packing qubits too close to each other, especially those involved in CNOT operations, can cause unwanted crosstalk that results in significantly degraded performace. On the other hand, packing qubits too far away from each other runs the risk of wasting robust qubits and creating unreliably long qubit SWAP paths.

Our proposed solution is to combine the strategies of creating a hierarchy tree for the available qubits [1] and utilizing adaptive multi-programming [2] to decide on a case-by-case basis whether the available qubits are robust enough to run concurrent circuits with ?

In [ ]:
#code

**References** (todo)
[1] 
[2] P. Das, S. S. Tannu, P. J. Nair, and M. Qureshi, “A Case for MultiProgramming Quantum Computers,” in Proceedings of the 52nd Annual IEEE/ACM 
    International Symposium on Microarchitecture (Micro), 2019.

https://arxiv.org/pdf/2207.14483
https://memlab.ece.gatech.edu/papers/MICRO_2019_4.pdf